Setup environment

In [ ]:
!pip install llama-cpp-python


Download model. For this example, we are using CodeLLAMA 13B Instruct.

In [ ]:
!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q6_K.gguf

# If you already have the model downloaded, or want to save it run the following sections.

Enable Drive Connection by Clicking the Button at Files at tab.

Save Model to Drive

In [ ]:
!cp ./mistral-7b-instruct-v0.2.Q6_K.gguf "/content/drive/MyDrive/LLAMA Models/"

Download Model from Drive

In [ ]:
!cp "/content/drive/MyDrive/LLAMA Models/mistral-7b-instruct-v0.2.Q6_K.gguf" ./mistral-7b-instruct-v0.2.Q6_K.gguf

Translator functions

In [ ]:
from llama_cpp import Llama, LlamaGrammar
llm = Llama(model_path="./mistral-7b-instruct-v0.2.Q6_K.gguf", n_ctx=2048, use_mlock=True)

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

def infer_translator(text):

    logging.info("Translating text: " + text)

    prompt = f'''[INST]<<SYS>>You are a professional translator.
You will first transliterate the Chinese sentences into Latin alphabet.
Then, you will translate the given Chinese sentences into English.
For example:
Chinese: 你好
Transliteration: nihao
English: Hello
[END]<</SYS>>

Here is the first sentence:
Chinese: {text}
Transliteration:[/INST]'''

    grammar_string = r'''# root rule defining the overall format of the answer
root ::= answer

# rule for the answer section
answer ::= transliteration "English: " english-text

# rule for the transliteration section
transliteration ::= "Transliteration: " trans-text "\n"

# rule for the English translation section
english-text ::= [^\n]+ "\n"

# rule to match any sequence of characters for the transliteration, excluding newline
trans-text ::= [^\n]+ "\n"

# Note: [^\n] denotes any character except a newline, and + indicates one or more repetitions.
'''
    grammar = LlamaGrammar.from_string(grammar_string)
    temperature = 0.2
    top_k = 40
    top_p = 0.9
    max_tokens = 256
    stop = ["[END]"]

    # response = llm(prompt=prompt, grammar=grammar, temperature=temperature, top_k=top_k, top_p=top_p, max_tokens=max_tokens, stop=stop)

    # response_content = response["choices"][0]["text"]

    # logging.info(response_content)

    response_iterator = llm(prompt=prompt, grammar=grammar, temperature=temperature, top_k=top_k, top_p=top_p, max_tokens=max_tokens, stop=stop, stream=True)

    return response_iterator

In [ ]:
infer_translator("你好")

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

# Text widget for input
input_text = widgets.Textarea(
    value='',
    placeholder='Text to translate',
    description='Input:',
    disabled=False,
    layout={'height': '100px', 'width': '600px'}
)

# Button to execute function
execute_button = widgets.Button(description="Translate")

progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)


# Textarea widgets to display the results
output1 = widgets.Textarea(
    value='',
    placeholder='Transliteration will be displayed here',
    description='Transliteration:',
    disabled=True,
    layout={'height': '200px', 'width': '600px'}  # Adjust height and width as needed
)

output2 = widgets.Textarea(
    value='',
    placeholder='Translation will be displayed here',
    description='Translation:',
    disabled=True,
    layout={'height': '200px', 'width': '600px'}  # Adjust height and width as needed
)

# Function to be executed
def execute_function(change):
    progress_bar.value = 0
    progress_bar.max = 3  # Set this to a high number for a smooth progress bar
    progress_bar.style.bar_color = 'blue'
    display(progress_bar)

    # Step 1: You might want to update progress bar here
    progress_bar.value += 1

    # output_text = infer_translator(input_text.value)

    for completion in infer_translator(input_text.value):
      completion_token = completion["choices"][0]["text"]
      output1.value += completion_token

    output_text = output1.value

    # Step 2: Another update to the progress bar
    progress_bar.value += 1

    parts = output_text.split('\n\n')
    transliteration_string = parts[0] if len(parts) > 0 else ""
    transliteration = transliteration_string.split('Transliteration:')[1]

    translation_string = parts[1] if len(parts) > 1 else ""
    translation = translation_string.split('English:')[1]

    output1.value = transliteration
    output2.value = translation

    # Step 3: Final update to the progress bar
    progress_bar.value += 1
    progress_bar.bar_style = 'success'

# Attach the function to the button
execute_button.on_click(execute_function)

# Display widgets
display(input_text, execute_button, output1, output2)


For example

https://shanghai.nyu.edu/news/nyu-shanghai-welcomes-class-2027

https://shanghai.nyu.edu/cn/news/nyu-shanghai-welcomes-class-2027

On the morning of August 21, NYU Shanghai welcomed more than 400 Chinese and international students from the Class of 2027 to the school’s eleventh University Welcome, the first to be held at its newly opened New Bund Campus. While students arrived on campus on August 20 for Move-In Day, the ceremony marked the first time the students joined together as an entire class.

2023年8月21日上午，上海纽约大学前滩校园热闹非凡，迎来了2023级本科生开学典礼。来自全球各地的400多名中外新生，在完成前一天的报到后，首次齐聚校园，正式开启了他们的大学之旅。